In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import LineString
import branca
import folium

## Import files

In [13]:
# Full database
# df is still the sample csv, need to be changed with the full dataset filtered on stations from Rotterdam to the Hague
df = pd.read_csv('prog en realisatie ophalen - sample1.csv') 
# Codes of stations associated to their commercial name
stations_codes = pd.read_excel("data_stations_codes.xlsx")
# GeoJson files that contains the geometry of the location of station
gdf_points = gpd.read_file("data_stations_geometry.geojson")

#Small database for testing
df= df.head(1000)

Associate each station to it's geometry

In [8]:
stations_codes = stations_codes.merge(
    gdf_points[["name","geometry"]], 
    left_on="Commercial Name", 
    right_on="name")
stations_codes = gpd.GeoDataFrame(stations_codes, geometry='geometry')

Split the fied TRAJECT in two separate fields to get Origin and Destination station, and to get their geometries

In [9]:
df["Origin_station"] = df["TRAJECT"].str.split("_").str[0]
df["Destination_station"] = df["TRAJECT"].str.split("_").str[1]

df = df.merge(
        stations_codes, 
        left_on="Origin_station",
        right_on = "Code",
        ).rename(columns = {
            "Commercial Name" : "Origin_com_name", 
            "Code" : "Origin_code",
            "geometry" : "geometry_origin"
            })
    
df = df.merge(
        stations_codes, 
        left_on="Destination_station",
        right_on = "Code",
        ).rename(columns = {
            "Commercial Name" : "Destin_com_name", 
            "Code" : "Destin_code",
            "geometry" : "geometry_destination"
            })


Create a table gdf_segments that contains all the informations about a segments (geometry, crowding)

In [11]:
gdf_segments = df[["TRAJECT","geometry_origin", "geometry_destination","Origin_com_name","Destin_com_name","Origin_code","Destin_code"]].drop_duplicates()
gdf_segments['geometry'] = gdf_segments.apply(
    lambda row: LineString([row['geometry_origin'], row['geometry_destination']]), 
    axis=1
)
gdf_segments = gpd.GeoDataFrame(gdf_segments, geometry='geometry')

crowding_by_traject = df.groupby("TRAJECT")["REALISATIE"].sum().reset_index()
gdf_segments = gdf_segments.merge(
    crowding_by_traject,
    left_on="TRAJECT",
    right_on="TRAJECT",
    how="left"
)
gdf_segments["crowding"] = gdf_segments["REALISATIE"]
gdf_segments = gdf_segments.set_crs("EPSG:4326", allow_override=True)


# Map

In [12]:
# create learn colormap interpolating 3 colors
colors = branca.colormap.LinearColormap(
    ['green', 'yellow', 'red'], vmin=gdf_segments["crowding"].min(), vmax=gdf_segments["crowding"].max())

def weight (val, min_weight, max_weight) : 
    vmin=gdf_segments["crowding"].min()
    vmax=gdf_segments["crowding"].max()
    norm = (val - vmin) / (vmax - vmin)
    return min_weight + norm * (max_weight - min_weight)

# define style function
def raster_choropleth(row):
    return {
        #"fillColor": colors(row['properties']['avg']),
        "color": colors(row['properties']['crowding']),
        "weight": weight (row['properties']['crowding'], 2, 5),
        "fillOpacity": 0.75,
    }

# create base map
# The map is centered on The Hague
poly_map = folium.Map(
        location=[52.05, 4.19],    
        zoom_start=7
    )

# overlay choropleth
gjson = folium.features.GeoJson(
    gdf_segments[['TRAJECT', 'crowding', 'geometry']],
    style_function=raster_choropleth,
    ).add_to(poly_map)

# add Tooltip
folium.features.GeoJsonTooltip(
    fields=['TRAJECT','crowding'],
    aliases=['Segment', 'Crowding']
).add_to(gjson)

# add colormap to the map
poly_map.add_child(colors)

# display
poly_map

Future improvements : 
- Divide the high-speed lines onto the intermediate segments to see the crowding on each proper segment
- Add more parameters : selection of the time interval to agregate on,....